# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

This is a Classification problem. Choosing between the class of needing intervention or not needing intervention which is a set of discrete labels. Regression is generally suitable for scenarios where the target label is a continuous variable, not a set of discrete values.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [36]:
# Read student data
student_data = pd.read_csv("student-data.csv", encoding="utf-8-sig")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns
print student_data.iloc[0]

Student data read successfully!
school             GP
sex                 F
age                18
address             U
famsize           GT3
Pstatus             A
Medu                4
Fedu                4
Mjob          at_home
Fjob          teacher
reason         course
guardian       mother
traveltime          2
studytime           2
failures            0
schoolsup         yes
famsup             no
paid               no
activities         no
nursery           yes
higher            yes
internet           no
romantic           no
famrel              4
freetime            3
goout               4
Dalc                1
Walc                1
health              3
absences            6
passed             no
Name: 0, dtype: object


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [159]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.iloc[1])
n_passed = ([x for x in student_data['passed']]).count('yes')
n_failed = ([x for x in student_data['passed']]).count('no')
grad_rate = float(float(n_passed)/n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [160]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
[u'school', u'sex', u'age', u'address', u'famsize', u'Pstatus', u'Medu', u'Fedu', u'Mjob', u'Fjob', u'reason', u'guardian', u'traveltime', u'studytime', u'failures', u'schoolsup', u'famsup', u'paid', u'activities', u'nursery', u'higher', u'internet', u'romantic', u'famrel', u'freetime', u'goout', u'Dalc', u'Walc', u'health', u'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4  

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [161]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
[u'school_GP', u'school_MS', u'sex_F', u'sex_M', u'age', u'address_R', u'address_U', u'famsize_GT3', u'famsize_LE3', u'Pstatus_A', u'Pstatus_T', u'Medu', u'Fedu', u'Mjob_at_home', u'Mjob_health', u'Mjob_other', u'Mjob_services', u'Mjob_teacher', u'Fjob_at_home', u'Fjob_health', u'Fjob_other', u'Fjob_services', u'Fjob_teacher', u'reason_course', u'reason_home', u'reason_other', u'reason_reputation', u'guardian_father', u'guardian_mother', u'guardian_other', u'traveltime', u'studytime', u'failures', u'schoolsup', u'famsup', u'paid', u'activities', u'nursery', u'higher', u'internet', u'romantic', u'famrel', u'freetime', u'goout', u'Dalc', u'Walc', u'health', u'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [265]:
from sklearn import cross_validation

# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size=0.24, random_state=74)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.


Note: You need to produce 3 such tables - one for each model.

Models chosen: 
- Decision Tree Classifier
    - Roughly O(2^N) time & space complexity
    * Decision Tree Classifiers are generally effective with large feature sets, have very fast query time and have been shown to empirically be effective. Decision trees are prone to overfitting.
      A decision tree classifier was a good model for establishing a ballpark predictive power for the data set, there is a fairly large number of features considering the size of the data set, and the decision tree takes no tuning to generate some predictive power.
- Gaussian Naive Bayes
    - Generally O(N) training time complexity and O(N) training space complexity, as well as O(1) query time & space.
    * Naive Bayes algorithms are very strong with a small dataset, and very efficient to both train and query. Naive Bayes approaches to estimatin (regression) are not particularly strong, but Gaussian Naive Bayes is particularly suited for this classification.
- Support Vector Machine
    - Generally O(n^2) training time & complexity, O(1) query complexity.
    * Support Vector Machines are generally effective with a large number of features, memory efficient and query efficient. SVMs are not particularly effective with smaller data sets however, which is a concern with this data set.
    The SVM conceptually seemed like it would provide predictive power around this data set, as covered in the description later on to the board.
    
Please see the results tables for the three models below just before section 5.
    
    




In [259]:
# Train a model

from sklearn import tree
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    return (end - start)


# TODO: Choose a model, import it and instantiate an object
from sklearn.ensemble import RandomForestClassifier
clf=tree.DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003


0.0025289058685302734

In [260]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return {'f1_score':f1_score(target.values, y_pred, pos_label='yes'),
            'prediction_time':(end - start)}

train_f1_score = predict_labels(clf, X_train, y_train)['f1_score']
print "F1 score for training set: {}".format(train_f1_score)


Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0


In [251]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)['f1_score'])

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.693548387097


In [261]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    time=train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train)['f1_score'])
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)['f1_score'])
    f1_score_and_predict_time=predict_labels(clf, X_train, y_train)
    return {'training_time':time, 'training_score':f1_score_and_predict_time['f1_score'],
            'test_score':predict_labels(clf, X_test, y_test)['f1_score'], 
            'prediction_time':f1_score_and_predict_time['prediction_time']}

train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)
# Note: Keep the test set constant

------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.612903225806
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.802919708029
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (sec

{'prediction_time': 0.0001590251922607422,
 'test_score': 0.8029197080291971,
 'training_score': 1.0,
 'training_time': 0.0006489753723144531}

In [266]:
from pandas import DataFrame
columns=[100, 200, 300]
index=['Training time (secs)', 'Prediction time (secs)', 'F1 score for training set', 'F1 score for test set']
dt_table_df = DataFrame(index=index, columns=columns)
nb_table_df = DataFrame(index=index, columns=columns)
svm_table_df = DataFrame(index=index, columns=columns)
vclf_table_df = DataFrame(index=index, columns=columns)
reg_vclf_table_df = DataFrame(index=index, columns=columns)

#for reference:
#print(train_predict(clf, X_train, y_train, X_test, y_test))
results=train_predict(clf, X_train, y_train, X_test, y_test)
dt_table_df.set_value('Training time (secs)', 300, results['training_time'])
dt_table_df.set_value('Prediction time (secs)', 300, results['prediction_time'])
dt_table_df.set_value('F1 score for training set', 300, results['training_score'])
dt_table_df.set_value('F1 score for test set', 300, results['test_score'])

results=train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
dt_table_df.set_value('Training time (secs)', 200, results['training_time'])
dt_table_df.set_value('Prediction time (secs)', 200, results['prediction_time'])
dt_table_df.set_value('F1 score for training set', 200, results['training_score'])
dt_table_df.set_value('F1 score for test set', 200, results['test_score'])

results=train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)
dt_table_df.set_value('Training time (secs)', 100, results['training_time'])
dt_table_df.set_value('Prediction time (secs)', 100, results['prediction_time'])
dt_table_df.set_value('F1 score for training set', 100, results['training_score'])
dt_table_df.set_value('F1 score for test set', 100, results['test_score'])

# TODO: Train and predict using two other model
#from sklearn.ensemble import AdaBoostClassifier

#clf2=AdaBoostClassifier(base_estimator=svm.SVC(probability=True, kernel='linear'))
#train_predict(clf2, X_train, y_train, X_test, y_test)
#train_predict(clf2, X_train[:200], y_train[:200], X_test, y_test)
#train_predict(clf2, X_train[:100], y_train[:100], X_test, y_test)


from sklearn.naive_bayes import GaussianNB

clf3=GaussianNB()
results=train_predict(clf3, X_train, y_train, X_test, y_test)
nb_table_df.set_value('Training time (secs)', 300, results['training_time'])
nb_table_df.set_value('Prediction time (secs)', 300, results['prediction_time'])
nb_table_df.set_value('F1 score for training set', 300, results['training_score'])
nb_table_df.set_value('F1 score for test set', 300, results['test_score'])

results=train_predict(clf3, X_train[:200], y_train[:200], X_test, y_test)
nb_table_df.set_value('Training time (secs)', 200, results['training_time'])
nb_table_df.set_value('Prediction time (secs)', 200, results['prediction_time'])
nb_table_df.set_value('F1 score for training set', 200, results['training_score'])
nb_table_df.set_value('F1 score for test set', 200, results['test_score'])

results=train_predict(clf3, X_train[:100], y_train[:100], X_test, y_test)
nb_table_df.set_value('Training time (secs)', 100, results['training_time'])
nb_table_df.set_value('Prediction time (secs)', 100, results['prediction_time'])
nb_table_df.set_value('F1 score for training set', 100, results['training_score'])
nb_table_df.set_value('F1 score for test set', 100, results['test_score'])


#from sklearn.ensemble import BaggingClassifier
#from sklearn.neighbors import KNeighborsClassifier

#clf4=BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
#train_predict(clf4, X_train, y_train, X_test, y_test)
#train_predict(clf4, X_train[:200], y_train[:200], X_test, y_test)
#train_predict(clf4, X_train[:100], y_train[:100], X_test, y_test)

from sklearn import svm
clf5=BaggingClassifier(svm.SVC(kernel='linear'), max_samples=0.5, max_features=0.5)
results=train_predict(clf5, X_train, y_train, X_test, y_test)
svm_table_df.set_value('Training time (secs)', 300, results['training_time'])
svm_table_df.set_value('Prediction time (secs)', 300, results['prediction_time'])
svm_table_df.set_value('F1 score for training set', 300, results['training_score'])
svm_table_df.set_value('F1 score for test set', 300, results['test_score'])

results=train_predict(clf5, X_train[:200], y_train[:200], X_test, y_test)
svm_table_df.set_value('Training time (secs)', 200, results['training_time'])
svm_table_df.set_value('Prediction time (secs)', 200, results['prediction_time'])
svm_table_df.set_value('F1 score for training set', 200, results['training_score'])
svm_table_df.set_value('F1 score for test set', 200, results['test_score'])

results=train_predict(clf5, X_train[:100], y_train[:100], X_test, y_test)
svm_table_df.set_value('Training time (secs)', 100, results['training_time'])
svm_table_df.set_value('Prediction time (secs)', 100, results['prediction_time'])
svm_table_df.set_value('F1 score for training set', 100, results['training_score'])
svm_table_df.set_value('F1 score for test set', 100, results['test_score'])

print("decision tree classifier: "+str(np.mean(cross_validation.cross_val_score(clf, X_train, y_train, cv=10, scoring='f1_weighted'))))
print("GaussianNB classifier: "+str(np.mean(cross_validation.cross_val_score(clf3, X_train, y_train, cv=10, scoring='f1_weighted'))))
print("Bagging Classifier with Support Vector Classifier: "+str(np.mean(cross_validation.cross_val_score(clf5, X_train, y_train, cv=10, scoring='f1_weighted'))))


------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.746031746032
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.71875
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.0

In [263]:
from IPython.display import display


display(str('Decision Tree Classifier'))
display(dt_table_df)

display(str('Gaussian Naive Bayes'))
display(nb_table_df)

display(str('Support Vector Machine'))
display(svm_table_df)

'Decision Tree Classifier'

,100,200,300
Training time (secs),0.000618935,0.00138807,0.00278211
Prediction time (secs),0.000136852,0.000849962,0.00033617
F1 score for training set,1,1,1
F1 score for test set,0.776119,0.703125,0.709677


'Gaussian Naive Bayes'

,100,200,300
Training time (secs),0.000669003,0.00113201,0.000749826
Prediction time (secs),0.00030899,0.000360966,0.000400782
F1 score for training set,0.821705,0.812261,0.785894
F1 score for test set,0.687023,0.682171,0.734375


'Support Vector Machine'

,100,200,300
Training time (secs),0.0410202,0.042959,0.0504229
Prediction time (secs),0.00298095,0.00593805,0.00936103
F1 score for training set,0.861314,0.848921,0.84058
F1 score for test set,0.729927,0.791367,0.723077


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

Notes: The performance of models varied significantly across different initial splits of data into training and test sets (changing the random_state seed on the initial cross_val separation of the data lead to dramatically different model performance).
 - The first seed I used 382953, lead to Gaussian NB outperforming all models, including the ensemble Voting Classifier tuned with Gridsearch that included GNB, SVM & Decision Tree classifiers. SVM generated significantly better results with the other two random_state seeds (74, 32) but generally possessed only as good an F1 score as the grid search tuned voting classifier. Because of the sparse data, I would be interested to see the results of a pure Gaussian Naive Bayes classifier over the long term with a larger dataset. I suspect this gives the highest predictive power with the greatest generality.
 
Explanation for Board:

A Support Vector Machine (SVM) learning algorithm gives us the best predictive power with the lowest computational costs. This finding from multiple model tests aligns with our expectations regarding the data set--- there will be students who, based on their attributes like household socio-economic status, shown in parents vs gaurdians, parent professins, study time, travel time we can be all but certain will pass or can be certain will struggle. In between these two different sets of features will be an area where our expectations are less clear, but at some point, there will be a shift in probability between passing and being unlikely to pass.

The SVM algorithm goes about identifying this line of demarcation by maximizing the amount of distance, or margin, of the separating the feature sets of the students who did pass and the feature sets of the students who did not. In doing so, we can then plot the feature sets of any new students, and based on their location, predict whether or not they are likely to pass. The SVM is relatively efficient to train, and very efficient to query which is desirable as we will query mre often than we will train (assuming we query for each student, but only train at the end of a semseter or other learning period.

------
Tuned F1 Score:

The tuned voting classifier had a final F1 score of .811, which peculiarly is less than the F1 score f1 score of the non grid search cv optimized voting classifier. As stated earlier, different random_state seeds for the cross validation split had wildly different performance, and the Gaussian Naive Bayes classifier is showing better performance on the untuned runs (but the voting classifier using both SVM, GNB & a decision tree is far outperforming either models). 

In [267]:
# Set up the parameters we wish to tune
parameters = {'voting':('soft', 'hard'), 'weights':(None, (1,2,1), (1,2,2), (2,2,1), (0,1,1), (0,1,0), (1,2,3), (3,2,1),(2,3,1), (2,1,3),(1,3,2), (1,0,1), (2,2,3), (1,1,2))}
#parameters = {'max_depth':(1,2,3,4,5,6,7,8,9,10)}

from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import GridSearchCV

vclf=VotingClassifier(estimators=[('rt', clf), ('nb', clf3), ('svm', clf5)], voting='soft', weights=[1,2,1])

results=train_predict(vclf, X_train, y_train, X_test, y_test)
vclf_table_df.set_value('Training time (secs)', 300, results['training_time'])
vclf_table_df.set_value('Prediction time (secs)', 300, results['prediction_time'])
vclf_table_df.set_value('F1 score for training set', 300, results['training_score'])
vclf_table_df.set_value('F1 score for test set', 300, results['test_score'])

results=train_predict(vclf, X_train[:200], y_train[:200], X_test, y_test)
vclf_table_df.set_value('Training time (secs)', 200, results['training_time'])
vclf_table_df.set_value('Prediction time (secs)', 200, results['prediction_time'])
vclf_table_df.set_value('F1 score for training set', 200, results['training_score'])
vclf_table_df.set_value('F1 score for test set', 200, results['test_score'])

results=train_predict(vclf, X_train[:100], y_train[:100], X_test, y_test)
vclf_table_df.set_value('Training time (secs)', 100, results['training_time'])
vclf_table_df.set_value('Prediction time (secs)', 100, results['prediction_time'])
vclf_table_df.set_value('F1 score for training set', 100, results['training_score'])
vclf_table_df.set_value('F1 score for test set', 100, results['test_score'])

reg = GridSearchCV(vclf, parameters, scoring='f1_weighted', cv=10)
results=train_predict(reg, X_train, y_train, X_test, y_test)
reg_vclf_table_df.set_value('Training time (secs)', 300, results['training_time'])
reg_vclf_table_df.set_value('Prediction time (secs)', 300, results['prediction_time'])
reg_vclf_table_df.set_value('F1 score for training set', 300, results['training_score'])
reg_vclf_table_df.set_value('F1 score for test set', 300, results['test_score'])

results=train_predict(reg, X_train[:200], y_train[:200], X_test, y_test)
reg_vclf_table_df.set_value('Training time (secs)', 200, results['training_time'])
reg_vclf_table_df.set_value('Prediction time (secs)', 200, results['prediction_time'])
reg_vclf_table_df.set_value('F1 score for training set', 200, results['training_score'])
reg_vclf_table_df.set_value('F1 score for test set', 200, results['test_score'])

results=train_predict(reg, X_train[:100], y_train[:100], X_test, y_test)
reg_vclf_table_df.set_value('Training time (secs)', 100, results['training_time'])
reg_vclf_table_df.set_value('Prediction time (secs)', 100, results['prediction_time'])
reg_vclf_table_df.set_value('F1 score for training set', 100, results['training_score'])
reg_vclf_table_df.set_value('F1 score for test set', 100, results['test_score'])

display(vclf_table_df)
display(reg_vclf_table_df)
display(reg.best_params_)





------------------------------------------
Training set size: 300
Training VotingClassifier...
Done!
Training time (secs): 0.087
Predicting labels using VotingClassifier...
Done!
Prediction time (secs): 0.014
F1 score for training set: 0.840579710145
Predicting labels using VotingClassifier...
Done!
Prediction time (secs): 0.006
F1 score for test set: 0.823529411765
Predicting labels using VotingClassifier...
Done!
Prediction time (secs): 0.013
Predicting labels using VotingClassifier...
Done!
Prediction time (secs): 0.005
------------------------------------------
Training set size: 200
Training VotingClassifier...
Done!
Training time (secs): 0.051
Predicting labels using VotingClassifier...
Done!
Prediction time (secs): 0.010
F1 score for training set: 0.840989399293
Predicting labels using VotingClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.781954887218
Predicting labels using VotingClassifier...
Done!
Prediction time (secs): 0.008
Predicting labels usin

,100,200,300
Training time (secs),0.047107,0.05073,0.0865681
Prediction time (secs),0.00703001,0.00805306,0.0126791
F1 score for training set,0.848101,0.840989,0.84058
F1 score for test set,0.866667,0.781955,0.823529


,100,200,300
Training time (secs),10.6254,15.7817,23.2603
Prediction time (secs),0.00586295,0.00810504,0.0139019
F1 score for training set,0.887417,0.901024,0.890951
F1 score for test set,0.870748,0.8,0.811189


{'voting': 'hard', 'weights': (2, 3, 1)}